# Clean up data, convert to dataframes, and save to csv

The BAC file is pipe separated with the following fields:
LogLevel [TimeStamp]:[LogVisibility][LogSeverity][entryType][entrySubType][eventType][EventType dependent strings]

The information changes at the end of each entry based on its EventType.  Here is a list of the supported event types and the subsequent additional information for each listed below.

GeneralMessage - [string message]

LevelChangedEvent - [load ID][loadName][roomName][rampTime][rampBaseValue][rampFinalValue]

ButtonChangedEvent - [keypad ID][keypadName][roomName][buttonNum][buttonState]

RemoteSystemEvent - [signalID][signalName][roomName][RemoteSystemEvent string]
TimeClockChangedEvent/OccupancyChangedEvent/SceneChangedEvent - [ID][name][roomName][message]

ConnectionStatus - [device ID][Name][room Name][connection status][Load 1 Room Name:Load 1 Name]|[Load 2 Room Name:Load 2 Name]

    NOTE: DeviceConnectionStatusWithOptions is the same format as ConnectionStatus. 
    
SignalChangedEvent - [device ID][Device Name][room Name][signal event ID][signal Value] - Signal event ID differs by device and signal value is either bool or int based on the eventID.

SignalChangedEventWithStrings - [device ID][Device Name][Signal Name][Signal Value string][Signal direction][message]


In [1]:
# FOR DATA ANALYSIS PUT HOUR, DAY, WEEK, MONTH, DATE, WEATHER, SUNRISE/SUNUP

In [3]:
import os 
import pandas as pd
import numpy as np
import h5py
from numpy import nan
from datetime import datetime, timedelta
from time import mktime
import os.path
pd.set_option('display.max_columns', 500)

/home/phcherne/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [13]:
def save_or_load_from_checkpoint(df, checkpoint_name):
    if not os.path.isfile(checkpoint_name):
        df.to_hdf(checkpoint_name, 'table', mode='w', append=True, complevel=9, complib='zlib', index=False)
    else:
        df = pd.read_hdf(checkpoint_name,'table')

In [4]:
labels = ['LogLevel',
          'TimeStamp',
          'LogVisibility',
          'LogSeverity',
          'entryType',
          'entrySubType',
          'eventType']
# from label list
EVENT_TYPE_INDEX = labels.index('eventType')

# from line in .bac file
LOG_LEVEL_START = 0
LOG_LEVEL_END = 3
TIMESTAMP_START = 6
TIMESTAMP_END = 14
PIPE_SEPARATED_DATA_START = 17

event_type_labels = [
    "string message", # GeneralMessage
    "load ID", "loadName", "roomName1", "rampTime", "rampBaseValue", "rampFinalValue", # LevelChangedEvent
    "keypad ID", "keypadName", "roomName2", "buttonNum", "buttonState", # ButtonChangedEvent
    "signalID", "signalName", "roomName3", "RemoteSystemEvent string", # RemoteSystemEvent
    "ID", "name", "roomName4", "message1", # TimeClockChangedEvent/OccupancyChangedEvent/SceneChangedEvent
    "device ID1", "Name", "roomName5", "connection status", "Load 1 Room Name:Load 1 Name", "Load 2 Room Name:Load 2 Name", # ConnectionStatus/DeviceConnectionStatusWithOptions
    "device ID2", "Device Name1", "roomName6", "signal event ID", "signal Value", # SignalChangedEvent
    "device ID3", "Device Name2", "Signal Name", "Signal Value string", "Signal direction", "message2", # SignalChangedEventWithStrings
]

def from_bac():
    # device_id_offsets = [i for i, x in enumerate(event_type_labels) if x == "device ID"]
    clean_lines = []

    for log in os.listdir('data'):
        with open('data/{}'.format(log)) as logfile:
            for line in logfile:
                line = line.rstrip('\n')
                if line[-1] == '|':
                    line = line[:-1]
                all_data = ([line[LOG_LEVEL_START:LOG_LEVEL_END], line[TIMESTAMP_START:TIMESTAMP_END]]
                            + line[PIPE_SEPARATED_DATA_START:].split('|'))

                event_type_dependent_strings = all_data[len(labels):]
                clean_line = all_data[:len(labels)]

                START_INDEX = 7
                if clean_line[EVENT_TYPE_INDEX] == 'GeneralMessage':
                    START_INDEX += event_type_labels.index('string message')
                    # account for pipes in the message string
                    event_type_dependent_strings = ['|'.join(event_type_dependent_strings)]
                    assert START_INDEX == 7
                    assert len(event_type_dependent_strings) == 1, event_type_dependent_strings

                elif clean_line[EVENT_TYPE_INDEX] == 'LevelChangedEvent':
                    START_INDEX += event_type_labels.index('load ID')
                    assert START_INDEX == 8
                    assert len(event_type_dependent_strings) == 6

                elif clean_line[EVENT_TYPE_INDEX] == 'ButtonChangedEvent':
                    START_INDEX += event_type_labels.index('keypad ID')
                    assert START_INDEX == 14
                    assert len(event_type_dependent_strings) == 5

                elif clean_line[EVENT_TYPE_INDEX] == 'RemoteSystemEvent':
                    START_INDEX += event_type_labels.index('signalID')
                    assert START_INDEX == 19
                    assert len(event_type_dependent_strings) == 4

                elif (clean_line[EVENT_TYPE_INDEX] == 'TimeClockChangedEvent' or
                      clean_line[EVENT_TYPE_INDEX] == 'OccupancyChangedEvent' or
                      clean_line[EVENT_TYPE_INDEX] == 'SceneChangedEvent'):
                    START_INDEX += event_type_labels.index('ID')
                    assert START_INDEX == 23
                    assert len(event_type_dependent_strings) == 4

                elif (clean_line[EVENT_TYPE_INDEX] == 'ConnectionStatus' or
                      clean_line[EVENT_TYPE_INDEX] == 'DeviceConnectionStatusWithOptions'):
                    START_INDEX += event_type_labels.index('device ID1')
                    assert START_INDEX == 27
                    assert (len(event_type_dependent_strings) == 4 or
                            len(event_type_dependent_strings) == 5 or
                            len(event_type_dependent_strings) == 6)

                elif clean_line[EVENT_TYPE_INDEX] == 'SignalChangedEvent':
                    START_INDEX += event_type_labels.index('device ID2')
                    assert START_INDEX == 33
                    assert len(event_type_dependent_strings) == 5

                elif clean_line[EVENT_TYPE_INDEX] == 'SignalChangedEventWithStrings':
                    START_INDEX += event_type_labels.index('device ID3')
                    assert START_INDEX == 38
                    # to correct for the double pipe in "Basement Mudroom"
                    event_type_dependent_strings = [i for i in event_type_dependent_strings if i]
                else:
                    raise ValueError("Wrong event type: {}".format(clean_line[EVENT_TYPE_INDEX]))

                clean_line = clean_line + [np.nan]*len(event_type_labels) + [log[-14:-4]]
                clean_line[START_INDEX:START_INDEX + len(event_type_dependent_strings)] = event_type_dependent_strings
                clean_lines.append(clean_line)
    df = pd.DataFrame(clean_lines, columns=labels + event_type_labels + ["date"])
    return df

## First Checkpoint: Load data from BAC files

In [5]:
if not os.path.isfile('./checkpoints/data_0.csv'):
    df = from_bac()
else:
    df = pd.read_csv('./checkpoints/data_0.csv')

KeyboardInterrupt: 

### Combine date and time

In [ ]:
def to_datetime(row):
    new_date = row['date'] + ' ' + row['TimeStamp']
    dt = datetime.strptime(new_date, '%Y-%m-%d %H:%M:%S')
    dt = dt + timedelta(hours=4)
    unix_secs = mktime(dt.timetuple())
    return unix_secs

In [ ]:
df['unix_time'] = df.apply(to_datetime, axis=1)
df = df.drop(["TimeStamp","date"], axis=1)

### Drop useless columns

In [ ]:
# Useless columns
df = df.drop(["LogLevel", "LogVisibility", "LogSeverity"],axis=1, errors='ignore')

# useless after deleting General Message
df = df.drop("string message", axis=1, errors='ignore')

# useless after deleting Button Change Event
df = df.drop(["keypad ID", "keypadName", "roomName2", "buttonNum", "buttonState"], axis=1, errors='ignore')

# useless after deleting Connection Status
df = df.drop(["device ID1", "Name", "roomName5", "connection status", "Load 1 Room Name:Load 1 Name", "Load 2 Room Name:Load 2 Name"], axis=1, errors='ignore')

# useless after deleting Remote System Event
df = df.drop(["signalID", "signalName", "roomName3", "RemoteSystemEvent string"], axis=1, errors='ignore')

### Remove useless event types

In [ ]:
df = df[df['eventType'] != 'ButtonChangedEvent']
df = df[df['eventType'] != 'GeneralMessage']
df = df[df['eventType'] != 'RemoteSystemEvent']
df = df[df['eventType'] != 'TimeClockChangedEvent']
df = df[df['eventType'] != 'ConnectionStatus']
df = df[df['eventType'] != 'DeviceConnectionStatusWithOptions']

### Merge Ids and Names

In [ ]:
def merge_columns(row, *args):
    count = sum(1 for column_name in args if pd.notna(row[column_name]))
    # checks if more than one value in the merged rows
    assert count < 2
    for column_name in args:
        if pd.notna(row[column_name]):
            return row[column_name]
    return np.nan

In [ ]:
# Merge room names
df['room_name_merged'] = df.apply(merge_columns, args=("roomName1","roomName4","roomName6"), axis=1)
df = df.drop(["roomName1","roomName4","roomName6"], axis=1)

In [ ]:
# Merge device name
df['device_name'] = df.apply(merge_columns, args=("loadName", "name", "Device Name1", "Device Name2"), axis=1)
df = df.drop(["loadName", "name", "Device Name1", "Device Name2"], axis=1)

In [ ]:
# Merge IDs 
df['device_id'] = df.apply(merge_columns, args=('load ID', 'ID', 'device ID2', 'device ID3'), axis=1)
df = df.drop(['load ID', 'ID', 'device ID2', 'device ID3'], axis=1)

In [ ]:
# Merge messages 
df['message_merged'] = df.apply(merge_columns, args=("message1", "message2"), axis=1)
df = df.drop(["message1", "message2"], axis=1)

### Combine device name and id to for truly unique ids

In [ ]:
df['device_id_name'] = df['device_id'].astype(str) + df['device_name'].astype(str)
df = df.drop(["device_id", "device_name"], axis=1)

In [ ]:
df = df.drop_duplicates()

In [ ]:
df = df.fillna('nan')

In [14]:
# Checkpoint 
save_or_load_from_checkpoint(df, './checkpoints/data_h5.h5')  

In [16]:
df = pd.read_hdf('./checkpoints/data_h5.h5','table')

In [17]:
df.head()

,entryType,entrySubType,eventType,rampTime,rampBaseValue,rampFinalValue,signal event ID,signal Value,Signal Name,Signal Value string,Signal direction,unix_time,room_name_merged,message_merged,device_id_name
1,Lights,Scene,SceneChangedEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.502438e+09,Upstairs Landing,SceneRecalled,74Night
3,DoorLock,Device,SignalChangedEventWithStrings,NaN,NaN,NaN,NaN,NaN,Basement Mudroom,Lock,ToDevice,1.502438e+09,NaN,NaN,208Basement Mudroom Door
4,DoorLock,Scene,SceneChangedEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.502438e+09,Basement Mudroom,SceneRecalled,77Lock Basement Mudroom Door
13,Lights,Load,LevelChangedEvent,0,0,19275,NaN,NaN,NaN,NaN,NaN,1.502438e+09,Upstairs Landing,NaN,118Sconces
14,Lights,Load,LevelChangedEvent,-1,0,0,NaN,NaN,NaN,NaN,NaN,1.502438e+09,Upstairs Landing,NaN,122Chandelier


### Normalize Column Values

In [18]:
for col in df.columns:
    print(col, df[col].unique())

entryType ['Lights' 'DoorLock' 'Climate' 'Auxiliary' 'System' 'Shades']
entrySubType ['Scene' 'Device' 'Load' 'Occupancy']
eventType ['SceneChangedEvent' 'SignalChangedEventWithStrings' 'LevelChangedEvent'
 'SignalChangedEvent' 'OccupancyChangedEvent']
rampTime [nan '0' '-1' ... '3122' '1655' '3273']
rampBaseValue [nan '0' '19275' ... '40828' '44286' '14330']
rampFinalValue [nan '19275' '0' '18247' '13621' '65535' '14135' '52685' '32639' '47031'
 '39064' '63222' '55512' '44244' '39567' '48385' '48156' '40597' '39590'
 '44267' '40617' '48169' '48398' '35723' '26471' '28784' '27756' '24929'
 '15163' '11051' '514' '38036' '13878' '29555' '35209' '33153' '31611'
 '33924' '17476' '22359' '21331' '4112' '22616' '2056' '22873' '8224'
 '7710' '3341' '64250' '63993' '17219' '7967' '32896' '44718' '21588'
 '24158' '14392' '2827' '8738' '28527' '22102' '17733' '24415' '19018'
 '9766' '13107' '27499' '11308' '6939' '28270' '51400' '20666' '26060'
 '20563' '20485' '17193' '26083' '20689' '17213' '2

Signal Name [nan 'Basement Mudroom' 'Master Bed' 'Great Room' 'Kitchen' 'Unassigned']
Signal Value string [nan 'Lock' 'Cool Stage 1' 'Active Fan Level' 'Temperature'
 'Scheduled Setpoints' '"Wake" Event' '"Leave" Event' '"Return" Event'
 '"Sleep" Event' 'Hold' 'Auto Setpoint' 'Cool Setpoint'
 '"Weekend Wake" Event' '"Wake Weekend" Event' 'Unlock' 'Heat Setpoint'
 'Heat Stage 1' 'Fan' 'Floor Warming' 'Humidifier Enable' 'Humidifier Off'
 'Slab 1' 'Slab 2' 'Slab 3' 'Slab 4A' 'Slab 4B' 'Slab 5A' 'Slab 5B'
 'Auto Mode' 'Single Setpoint Mode' 'Heat Mode' 'Cool Mode'
 'Humidity Mode' 'Humidity View' 'Auto Deadband' 'Mode' 'Slab Setpoint'
 'Humidity Setpoint' 'Get Information' 'Slab Temperature']
Signal direction [nan 'ToDevice' 'Inactive' 'Off' '71° Fahrenheit' '72° Fahrenheit'
 '74° Fahrenheit' '75° Fahrenheit' '73° Fahrenheit' 'Active' 'High'
 'Heat = 71° Fahrenheit, Cool = 75° Fahrenheit, and Auto = 72° Fahrenheit'
 'Heat Setpoint 71° Fahrenheit, Cool Setpoint 75° Fahrenheit, and Auto Set

In [19]:
df.loc[df["eventType"] == "SignalChangedEventWithStrings"]["entryType"].unique()

array(['DoorLock', 'Climate'], dtype=object)

In [20]:
# dropping more useless data
index_to_drop = df.loc[df["entryType"] == "Auxiliary"][df['eventType'] == "SignalChangedEventWithStrings"].index.tolist()
df = df.drop(index_to_drop)

/home/phcherne/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [ ]:
def parse_setpoints(row):
    heat_cool_auto = row['Signal direction'].split(',')
    heat_temp = [int(s) for s in heat_cool_auto[0].split() if s.isdigit()]
    cool_temp = [int(s) for s in heat_cool_auto[1].split() if s.isdigit()]
    auto_temp = [int(s) for s in heat_cool_auto[2].split() if s.isdigit()]
    return heat_temp, cool_temp, auto_temp

df['HeatSetpoint'], df['CoolSetpoint'], df['AutoSetpoint'] = df[df['Signal direction'].str.len() > 20].apply(parse_setpoints, axis=1)

In [ ]:
df.head(20)

### Encode Columns

Convert Categorical to Numerical Data. 

In [ ]:
from sklearn import preprocessing
columns_to_update = ["entryType", "entrySubType", "device_id_name", "eventType", "Signal Name", "Signal Value string", "Signal direction", "room_name_merged", "message_merged"]

def encode_columns(data_frame, column_names):
    label_encoders = {}
    for col in column_names:
        values = data_frame[col].unique()
        le = preprocessing.LabelEncoder()
        le.fit(values)
        label_encoders[col] = le
        numerical_values = le.transform(data_frame[col])
        data_frame[col] = numerical_values
    return data_frame, label_encoders

categorical_df = df.copy()
categorical_df, label_encoders = encode_columns(categorical_df, columns_to_update)